<a href="https://colab.research.google.com/github/kurehajime/openapi_sandbox/blob/main/HackerNews%E3%82%92%E6%97%A5%E6%9C%AC%E8%AA%9E%E3%81%A7%E8%AA%AD%E3%81%BF%E3%81%9F%E3%81%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下の入力ボックスにOpenAIのAPIキーを入力してください。

In [13]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [8]:
!pip install feedparser
!pip install tiktoken
!pip install newspaper3k
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=ad472807f1c6a5601e2c7c78ce8e74f3d30b9ba1a02b61bfa667eac715a3b400
  Stored in directory: /root/.cache/pip/wheels/94/ad/df/a2a01300cea47d5695f242f7e925a805970106fd9e4b151468
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-a

In [15]:
import feedparser
def get_entries(url):
  feed = feedparser.parse(url)
  entries = []
  for entry in feed['entries']:
    entries.append({
        "title":entry['title'],
        "link":entry['link'],
        "comments":entry['comments'],
    })
  return entries

In [18]:
from newspaper import Article
def get_news(url):
  article = Article(url)
  article.download()
  article.parse()
  return article.text

In [19]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
def get_head_by_token(text,head_size,sep):
  lines = text.split(sep)
  blocks = []
  token = 0
  block = ''
  for line in lines:
    t = len(encoding.encode(line))
    if token > 0 and head_size < (token + t):
      return block
    token += t
    block += line
  return block

In [26]:
import openai
url = "https://hnrss.org/newest?points=100"
entries = get_entries(url)
openai.api_key = secret
output = ""
for entry in entries[:5]:
  text = get_news(entry["link"])
  head = get_head_by_token(text,1000,"\n")
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "user", "content": f"""
以下の文章を日本語に翻訳して3行に要約してください。
---
{head}
          """},
      ]
  )
  sumally = ""
  for cho in completion.choices:
    sumally += cho.message.content
    output += f"""
### [{entry["title"]}]({entry["comments"]})
{sumally}
    """
  
from IPython.display import display, Markdown
display(Markdown(output))



### [Google discontinues Google Glass for enterprise](https://news.ycombinator.com/item?id=35178986)


「10年以上にわたる革新とパートナーシップに感謝します。2023年3月15日をもって、Glass Enterprise Editionの販売を終了します。2023年9月15日までは引き続きサポートを提供します。詳細はヘルプセンターをご覧ください。」

- Glass Enterprise Editionの販売終了（2023年3月15日）
- サポートは2023年9月15日まで提供
- 詳細はヘルプセンターを確認
    
### [GPT-4 Designed a Programming Language](https://news.ycombinator.com/item?id=35177857)
GPT-4がリリースされ、AI競争が加速している中、多くの人々は興奮と恐怖を感じている。GPT-4は専門家の中でも優秀な人々が驚くほど素晴らしいものを生み出しており、その進化はとどまるところを知らない。この進化を楽しむものと恐れるものがある中、筆者はGPT-4のチャットのベータ版に選ばれ、4時までプレイしていた。

GPT-4のリリースにより、AI競争は加速する一方であり、それは喜びと恐怖を共有する人々に影響を与えている。筆者自身もGPT-4のチャットのベータ版に選ばれ、様々なプログラム言語について学んだが、GPT-4はプログラム言語を設計するという高度な課題にも取り組んでいる。
    
### [Will AIs take all our jobs and end human history? It’s complicated](https://news.ycombinator.com/item?id=35177257)
文章ではAIのChatGPTについて説明し、人間によって書かれたテキストから学び、人間に似たエッセイを自動生成できることを示しています。ChatGPTは「共有コンテキスト」を基に、文字のプロンプトに基づいてエッセイを生成することができます。この技術により、人間が行っていた翻訳やエッセイ作成などの作業が自動化されることが期待されます。
    
### [Fireball Spotted over Northeastern USA](https://news.ycombinator.com/item?id=35175805)
The company has been working on a new product for the past year, and they are finally ready to launch it into the market. The product is expected to be a game-changer in its industry and the company is confident in its success. The launch date is set for next month and the company is already receiving pre-orders from interested customers.

1.新製品の開発が完了し、来月の発売日を迎える予定。
2.業界に革命をもたらすと期待され、注文も既に受け付け中。
3.会社は成功を確信している。
    
### [Emulating Pokemon Emerald on GPT-4](https://news.ycombinator.com/item?id=35175321)
JavaScriptが利用できません。このブラウザではJavaScriptが無効になっていることが検出されました。続けるにはJavaScriptを有効にするか、サポートされているブラウザに切り替えてください。サポートされているブラウザのリストは、ヘルプセンターで確認できます。 

JavaScriptが無効なため、Twitter.comの利用ができない。ブラウザを切り替えてあるいはJavaScriptを有効にしてください。
    